# openCV 作業：
* 題目 ：請找一段約一分鐘左右的影片，利用上課時學過的影像處理功能，發揮所有創意和想像，將該影片加工特效，產生一段新影片，怕誤解同學的創意，請將你的亮點盡可能寫在影片中的右下角

* 上傳 ：導師會有告訴同學上傳空間, 或可上傳到Youtube 或自己的雲端空間, 把連結給老師(依導師規範為準)

In [30]:
import cv2
import numpy as np

def add_text(frame, text):
    font = cv2.FONT_HERSHEY_SIMPLEX
    position = (10, 50)
    font_scale = 1
    font_color = (0, 255, 255)  # Yellow color
    thickness = 3  # Bold text
    cv2.putText(frame, text, position, font, font_scale, font_color, thickness, cv2.LINE_AA)

def apply_effect(frame, effect_name):
    if effect_name == "Grayscale":
        return cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    elif effect_name == "Sepia":
        sepia_filter = np.array([[0.272, 0.534, 0.131],
                                 [0.349, 0.686, 0.168],
                                 [0.393, 0.769, 0.189]])
        sepia_frame = cv2.transform(frame, sepia_filter)
        return np.clip(sepia_frame, 0, 255).astype(np.uint8)
    elif effect_name == "Invert":
        return cv2.bitwise_not(frame)
    elif effect_name == "Edge Detection":
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        edges = cv2.Canny(gray_frame, 100, 200)  # Strongest edge detection
        return cv2.cvtColor(edges, cv2.COLOR_GRAY2BGR)
    elif effect_name == "Gaussian Blur":
        return cv2.GaussianBlur(frame, (25, 25), 0)  # Enhanced blur effect
    elif effect_name == "Sobel":
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        grad_x = cv2.Sobel(gray_frame, cv2.CV_64F, 1, 0, ksize=3)
        grad_y = cv2.Sobel(gray_frame, cv2.CV_64F, 0, 1, ksize=3)
        grad = cv2.magnitude(grad_x, grad_y)
        sobel_frame = np.uint8(np.clip(grad, 0, 255))
        return cv2.cvtColor(sobel_frame, cv2.COLOR_GRAY2BGR)
    else:
        return frame

def main(input_video, output_video):
    cap = cv2.VideoCapture(input_video)
    if not cap.isOpened():
        print("Error: Cannot open video file.")
        return

    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

    effects = ["Grayscale", "Sepia", "Invert", "Edge Detection", "Gaussian Blur", "Sobel"]
    effect_duration = 10 * fps
    total_duration = 6 * effect_duration  # 6 effects, each lasting 10 seconds

    frame_count = 0
    while frame_count < total_duration:
        ret, frame = cap.read()
        if not ret:
            cap.set(cv2.CAP_PROP_POS_FRAMES, 0)  # Loop the video if it ends
            continue

        current_effect = effects[(frame_count // effect_duration) % len(effects)]
        processed_frame = apply_effect(frame, current_effect)

        if current_effect == "Grayscale":
            processed_frame = cv2.cvtColor(processed_frame, cv2.COLOR_GRAY2BGR)

        add_text(processed_frame, f"Effect: {current_effect}")
        out.write(processed_frame)

        frame_count += 1

    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    input_video = r"C:\Users\David\Desktop\Rick.mp4"  # 替換成您的影片路徑
    output_video = r"C:\Users\David\Desktop\Rick_out.mp4"  # 輸出影片路徑
    main(input_video, output_video)
